# Third parties exporting

## Token genaration and api management

Token genaration and api management


In [17]:
#@title Import modules
import requests, base64
import re
import json
import pandas as pd
from io import StringIO

In [18]:
#@title Token generation
#@markdown This code calls the token end point with client-id and secret

def login():
    cfg = pd.read_csv("/content/config.csv")
    client_id = cfg['client_id'].values[0]
    client_secret = cfg['client_secret'].values[0]
    token_endpoint = cfg['token_url'].values[0]
    
    data = {'grant_type': 'client_credentials'}
    
    userpass = client_id + ':' + client_secret
    encoded_u = base64.b64encode(userpass.encode()).decode()
    auth_header = "Basic " + encoded_u
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': auth_header
    }

    r = requests.post(token_endpoint, headers=headers, data=data)
    response = r.json()

    if (r.status_code == 200):
        token = response['access_token']
        # Put token in the session
        print(token)
    return token

In [19]:
#@title Get Results 
#@markdown This code calls the end point to retrieve data
def get_results(token, request):
    try:
        headers = {"Authorization": "Bearer " + token}
        cfg = pd.read_csv("config.csv")
        base_url = cfg['base_url'].values[0]
        result = requests.get(base_url + request, headers=headers)
        if 200 <= result.status_code < 300:
          return result.text
        else:
          print(result.text)
          return  
    except:
        return

In [20]:
#@title Post results  
#@markdown This code allows to POST data in payload or as a stream
def post_results(token, request, payload, files):
    try:
        headers = {"Authorization": "Bearer " + token}
        cfg = pd.read_csv("config.csv")
        base_url = cfg['base_url'].values[0]
        result = requests.post(base_url + request, headers=headers, data=payload, files = files)
        if 200 <= result.status_code < 300:
          json_data = json.loads(result.text)
          return json_data
        else:
          print(result.text)
          return
    except:
        return

In [21]:
#@title Launch report
#@markdown Code to manage report launch and retrieval of data in one function
#@markdown It launchs the task and waits until completion
def run_report(token, report):
    try:
        result = post_results(token, '/v1/process-templates/'+ report +'/run',"","")
        #pprint (result,indent=1)
        taskId = result[0]['taskId']
        while True:
            result = get_results(token, '/v1/process-templates/'+ taskId +'/status')
            if result == "Pending" or result == "In progress":
                print ('.', end='')
            else:
                print ('\n' + result)
            if result == "Warning" or result == "Complete" or result == "Error":
                break
                time.sleep(1)
        print('\n')
        result = get_results(token, '/v1/process-templates/'+ report + '/files?taskId=' + taskId )
        #print(result)
        return result
    except:
        return

## Connection

In [22]:
#@title <font color='lime'>Connect</font>
print('token:')
token = login()

token:
27d2f904-6ef9-4f2c-a5b2-017308f20e6a


## Exporting third-parties

In [25]:
#@title <font color='lime'>Launch report on third-parties</font>
res = run_report(token,'SU11')
buffer = StringIO(res)
df = pd.read_csv (buffer, header=None)
pd.set_option('display.max_colwidth', None)
df

......
Complete




,0
0,TDEMO_SUPPLIER Demo supplier for template 11 1 0 FR 0 EUR 0.000 0
1,ADEMO_SUPPLIER FR 1 EUR1FR1920041100200058741005T15 01 0 0 DEMO_BUYER 1 1 0
2,TFR001 FR Third Party 01 11 1 0 FR 0 EUR 0.000 0
3,AFR001 FR BNP Paribas 1 EUR300000000000000000000000 01 0 0 1 1 0
4,TFR002 FR Third Party 02 11 1 0 FR 0 EUR 0.000 0
5,TGB001 GB Third Party 01 11 1 0 GB 0 EUR 0.000 0
6,TGB002 GB Third Party 2 11 1 0 GB 0 EUR 0.000 0
7,TJP001 JP Third Party 01 11 1 0 JP 0 EUR 0.000 0
8,TJP002 JP Third Party 02 11 1 0 JP 0 EUR 0.000 0
9,TTEST034 Test Kyriba 034 11 1 0 75001 FR 0 EUR 0.000 0
